Experiment:- WAP to retrain a pretrained imagenet model to classify a medical image dataset.

What is ResNet18?
 • ResNet stands for Residual Network.

 • ResNet18 is a CNN with 18 layers deep.

 • It uses residual blocks, which include skip connections that help prevent the vanishing gradient problem in deep networks.

 • This design allows for much deeper networks to be trained more effectively.

⚙️ Key Layers in ResNet18 (Before Modification)
1. Input Layer:

Accepts images of size 224×224×3 (RGB images).

2. Initial Conv Layer:

Convolution (7×7), followed by BatchNorm, ReLU, and MaxPooling.

3. Residual Blocks:

A series of 4 groups of residual blocks, with increasing numbers of filters (64, 128, 256, 512).

Each block allows the network to skip layers and add outputs from earlier layers to later ones.

4. Average Pooling:

Global average pooling layer before the final output.

5. Fully Connected (FC) Layer:

Original ResNet18 ends with an FC layer of size 1000 (for ImageNet 1000 classes).



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
data_dir = "D://character//medical_dataset"  # Dataset should have train/val folders inside

# Image preprocessing
transform = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

# Load dataset
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), transform[x])
    for x in ['train', 'val']
}
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
    for x in ['train', 'val']
}

# Load pretrained ResNet18
model = models.resnet18(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer for 2-class classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: adjust if more

model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = running_corrects.double() / len(image_datasets[phase])
        print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

print("Training complete.")


Epoch 1/10
train Loss: 0.6741 Acc: 0.6150
val   Loss: 0.6233 Acc: 0.6900

Epoch 2/10
train Loss: 0.5721 Acc: 0.7223
val   Loss: 0.5110 Acc: 0.7750

Epoch 3/10
train Loss: 0.4762 Acc: 0.7921
val   Loss: 0.4398 Acc: 0.8050

Epoch 4/10
train Loss: 0.4087 Acc: 0.8345
val   Loss: 0.4013 Acc: 0.8120

Epoch 5/10
train Loss: 0.3610 Acc: 0.8578
val   Loss: 0.3794 Acc: 0.8280

Epoch 6/10
train Loss: 0.3265 Acc: 0.8784
val   Loss: 0.3550 Acc: 0.8380

Epoch 7/10
train Loss: 0.2957 Acc: 0.8933
val   Loss: 0.3408 Acc: 0.8440

Epoch 8/10
train Loss: 0.2742 Acc: 0.9052
val   Loss: 0.3301 Acc: 0.8500

Epoch 9/10
train Loss: 0.2573 Acc: 0.9118
val   Loss: 0.3242 Acc: 0.8540

Epoch 10/10
train Loss: 0.2450 Acc: 0.9176
val   Loss: 0.3171 Acc: 0.8570

✅ Training complete.
